In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import torch
from torch.utils import data
from torch import nn, optim
from sklearn import model_selection, preprocessing, metrics
import joblib
import albumentations as A
import torchvision

## Download dataset

In [ ]:
os.environ['KAGGLE_USERNAME']=''
os.environ['KAGGLE_KEY']=''

In [ ]:
!pip install kaggle 
!kaggle datasets download -d zippyz/cats-and-dogs-breeds-classification-oxford-dataset
!unzip -q cat*


 98% 768M/780M [00:06<00:00, 169MB/s]
100% 780M/780M [00:06<00:00, 131MB/s]


## Making a dataframe with image paths and labels

In [ ]:
img_root_dir = Path('images')
img_paths = list(img_root_dir.glob('**/*.jpg'))
labels = list(map(lambda path_: os.path.split(path_)[1], img_paths))
labels = list(map(lambda path: path.rsplit('_', 1)[-2], labels))
df = pd.DataFrame({'path': img_paths, 'labels': labels})

In [ ]:
mask = df['labels'].str[0].str.isupper()
df_final = df.loc[mask].copy()

In [ ]:
df_final['labels'] = df_final['labels'].str.replace('_', ' ')

In [ ]:
lbl_encoder = preprocessing.LabelEncoder()
labels = lbl_encoder.fit_transform(df_final['labels'])

In [ ]:
df_final['labels'] = labels
df_final.reset_index(inplace = True, drop = True)

## cloning my pytorch framework, for a more 'clean' experience

In [ ]:
!git clone -b testing https://github.com/default-303/easyTorch.git

Cloning into 'easyTorch'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 68 (delta 20), reused 65 (delta 17), pack-reused 0
Unpacking objects: 100% (68/68), done.


## Installing albumentations, a library I use for image transforms

In [ ]:
!pip install albumentations

     |████████████████████████████████| 631 kB 5.4 MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-py3-none-any.whl size=654017 sha256=44f4b7371e6f95c9639b82f7bfaa48b120d7d9ac933f720900a3b756e16b59de
  Stored in directory: /root/.cache/pip/wheels/89/72/98/3ebfdba1069a9a8eaaa7ae7265cfd67d63ef0197aaee2e5f9c
Successfully built imgaug
  Attempting uninstall: imgaug
    Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [ ]:
BATCH_SIZE = 224
EPOCHS = 10
LEARNING_RATE = 0.01
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
image_transforms = {
    'train': A.Compose([
        A.Resize(224, 224),
        A.HorizontalFlip(p = 0.6),
        A.Rotate([-40, 40], border_mode = cv2.BORDER_CONSTANT)
    ]), 
    'val': A.Compose([
        A.Resize(224, 224)
    ])
}

## Splitting the dataset

In [ ]:
train_index, test_index = model_selection.train_test_split(df_final.index.values, test_size = 0.1, shuffle = True, stratify = df_final['labels'])

## Initializing Datasets and Dataloader 

In [ ]:
datasets = {
    'train': imageDataset.CustomDataset(df_final.loc[train_index], image_transforms['val'], exit_on_error = True, random_on_error = False),
    'val': imageDataset.CustomDataset(df_final.loc[test_index], image_transforms['val'], exit_on_error = True, random_on_error = False),
}

In [ ]:
samplers = {
    'train': data.SubsetRandomSampler(np.arange(len(train_index))),
    'val': data.SubsetRandomSampler(np.arange(len(test_index)))
}

dataloaders = {
    'train': data.DataLoader(datasets['train'], batch_size = BATCH_SIZE, sampler = samplers['train']),
    'val': data.DataLoader(datasets['val'], batch_size = BATCH_SIZE, sampler = samplers['val'])
}

## Initialzing and modifying densenet

In [ ]:
import torchvision
densenet = torchvision.models.densenet121(pretrained = True)

for param in densenet.parameters():
    param.requires_grad = False

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

In [ ]:
import torch.nn.functional as F
class ModifiedPretrained(nn.Module):
    def __init__(self, pretrained_model):
        super().__init__()
        self.modifiedModel = pretrained_model
        in_features = self.modifiedModel.classifier.in_features
        model = nn.Sequential(
            nn.Linear(in_features, 512), 
            nn.ReLU(),
            nn.Linear(512, 128), 
            nn.ReLU(),
            nn.Linear(128, 12)
        )

        self.modifiedModel.classifier = model

        
    
    def forward(self, input):
        logits = self.modifiedModel(input)
        return logits


modfiedDNet = ModifiedPretrained(densenet)

modfiedDNet = modfiedDNet.to(device)



In [ ]:
optimizer = optim.Adam(modfiedDNet.parameters(), lr = LEARNING_RATE)
loss_func = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 5, factor = 0.1, verbose = True)
metrics_ = [(metrics.recall_score, {'average': 'micro'}),(metrics.recall_score, {'average': 'macro'})]

## Training

In [ ]:
model = trainer.Trainer(modfiedDNet,metrics_ , loss_func, optimizer, scheduler)

In [ ]:
model.fit(dataloaders)

[Train]Epoch [0/10]: 100%|██████████| 10/10 [00:29<00:00,  2.94s/it]



 running loss: 3.130086287745723, running acc: 0.11666666666666667

recall_score: 0.11666666666666667

recall_score: 0.11666666666666665


[VAL] Epoch [0/10]:  50%|█████     | 1/2 [00:02<00:02,  2.93s/it]


 running loss: 2.1162364641825357


[VAL] Epoch [0/10]: 100%|██████████| 2/2 [00:03<00:00,  1.59s/it]


...


 running loss: 0.18297247494812366, running acc: 0.9361111111111111

recall_score: 0.9361111111111111

recall_score: 0.9361111111111112


[VAL] Epoch [9/10]:  50%|█████     | 1/2 [00:02<00:02,  2.87s/it]


 running loss: 0.41937811772028605


[VAL] Epoch [9/10]: 100%|██████████| 2/2 [00:03<00:00,  1.55s/it]


 running loss: 0.4385028521219889

recall_score: 0.875

recall_score: 0.875


## Saving model state dict (and label encoder) for future use

In [ ]:
torch.save(modfiedDNet.state_dict(), './iniModel.pth')

In [ ]:
joblib.dump(lbl_encoder, 'labelEncoder.joblib')

['labelEncoder.joblib']

## Predict function to take care of image preprocessing and predicition

In [ ]:
def predict(model, image_path):
    img = io.imread(image_path)
    img = transform(image = img)['image']
    img = torch.tensor(img, dtype = torch.float)
    img = torch.permute(img, (2, 0, 1))
    img = torch.unsqueeze(img, 0)
    logits = model(img.cuda())
    pred = F.softmax(logits, 1)
    print(pred)
    return torch.argmax(pred, 1)
    
    
predict(modfiedDNet, 'aby.png')